# NFNet Inference


In [1]:
import numpy as np
import cv2
import json
import time
import os.path
from os import path 
import sys

### Importing MIGraphX Library
Sometimes the PYTHONPATH variable is not set during installation of MIGraphX. If your receive a "Module Not Found" error when trying to `import migraphx` in your own application, try running:
```
$ export PYTHONPATH=/opt/rocm/lib:$PYTHONPATH
```
For this example, the library will be added to the kernel's sys.path.

In [2]:
migx_lib_path = "/opt/rocm/lib"
if migx_lib_path not in sys.path:
    sys.path.append(migx_lib_path)

In [3]:
import migraphx

### Create NFNet ONNX file

In [4]:
!git clone https://github.com/cagery/pytorch-image-models.git

fatal: destination path 'pytorch-image-models' already exists and is not an empty directory.


In [5]:
!python3 ./pytorch-image-models/pt_to_onnx.py

/cgMIGraphX/nfnet/AMDMIGraphX/examples/python_nfnet_inference/pytorch-image-models/timm/models/layers/padding.py:19: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return max((math.ceil(x / s) - 1) * s + (k - 1) * d + 1 - x, 0)
/cgMIGraphX/nfnet/AMDMIGraphX/examples/python_nfnet_inference/pytorch-image-models/timm/models/layers/padding.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return max((math.ceil(x / s) - 1) * s + (k - 1) * d + 1 - x, 0)
/cgMIGraphX/nfnet/AMDMIGraphX/examples/python_nfnet_inference/pytorch-image-models/timm/models/layers/padd

In [6]:
### Load ONNX model using MIGraphX

In [9]:
model = migraphx.parse_onnx("dm_nfnet_f0.onnx")
model.compile(migraphx.get_target("gpu"))
model.print()     # Printed in terminal 

RuntimeError: /cgMIGraphX/nfnet/AMDMIGraphX/src/onnx/parse_slice.cpp:33: parse: PARSE_SLICE: cannot handle step other than 1

### Download a sample image

In [ ]:
!wget https://live.staticflickr.com/65535/50594927526_f6c3b2a5d4_b.jpg -O peppers.jpg
im = Image.open('peppers.jpg')
# Resize and crop to variant test size
imsize = base.nfnet_params[variant]['test_imsize']
im = im.resize((imsize + 32, imsize + 32))
im = im.crop((16, 16, 16+imsize, 16+imsize))
# Convert im to tensor and normalize with channel-wise RGB
MEAN_RGB = (0.485 * 255, 0.456 * 255, 0.406 * 255)
STDDEV_RGB = (0.229 * 255, 0.224 * 255, 0.225 * 255)
x = (np.float32(im) - MEAN_RGB) / STDDEV_RGB